## RDD example

This notebook provide a dummy example of a map on Spark RDD, that can be used to check that parallelisation works fine on the cluster.

It consist of creating an RDD with `n_partitions` partitions, and apply a map function that waits for 2 seconds for each partition.

## Preliminaries: Cluster access

* Connect to cluster with 

```
ssh -p 30 -L 8000:jupyter:8000 -L 8888:hue:8888 -L 8088:hue:8088 yourlogin@bigdata.ulb.ac.be
```

Note the port redirection to get access locally to JupyterHub, Hue, and Hadoop Web UI

* Open JupyterHub locally by connecting to `127.0.0.1:8000`
* Upload this notebook


#### General imports

In [15]:
import os 
import time

#### Start Spark session

A Spark session is created by using the pyspark.sql.SparkSession object. See [here](https://spark.apache.org/docs/latest/sql-programming-guide.html#starting-point-sparksession) for the API documentation on the SparkSession Object. 


In [1]:
#This is needed to start a Spark session from the notebook
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.ui.port=4050 \
                                    --num-executors 5 \
                                    --conf spark.driver.memory=2g\
                                    pyspark-shell"

#This is needed if running with Yarn from the cluster
os.environ['HADOOP_CONF_DIR'] = "/etc/hadoop/conf"
os.environ['PYSPARK_PYTHON']="/etc/anaconda3/bin/python"
os.environ['PYSPARK_DRIVER_PYTHON']="/etc/anaconda3/bin/python"


from pyspark.sql import SparkSession

NameError: name 'os' is not defined

In [21]:
#Uncomment below to recreate a Spark session with other parameters
spark.stop()
spark = SparkSession \
    .builder \
    .master("yarn") \
    .config("spark.executor.instances","10") \
    .appName("demoRDD") \
    .getOrCreate()
    
#When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
sc=spark.sparkContext

#### Start dummy Spark jobs

In [22]:
# Wait function
def wait2s(x):
    time.sleep(2)
    return x

In [23]:
n_partitions=8

data=range(0,n_partitions)
datardd=sc.parallelize(data,n_partitions)

datardd.map(wait2s).collect()

[0, 1, 2, 3, 4, 5, 6, 7]

### Open Spark UI and check parallelisation

* Open Hadoop Web UI at `127.0.0.1:8088`, and click on your running Application. You should land on an URL similar to `http://127.0.0.1:8088/cluster/app/application_1523870291186_0032`
* Change `cluster/app` to `proxy` to get to the Spark UI: `http://127.0.0.1:8088/proxy/application_1523870291186_003`